# SEO Analytics Overview
### Situation:
We are tasked with analyzing a website that publishes blogs about data analytics tools and working mindset. The website currently suffers from data quality issues, where the data is misplaced and not in a clean format. Our objective is to clean this data and provide a cleaned CSV file output that can be further analyzed to gain insights.
### Steps to Achieve the Objective:
## 1. Data Collection and Initial Inspection:
- Gather raw data from crawling the website
- Perform inspection to identify issues such as missing values, misplaced data, and inconsistencies in formatting.
## 2. Data Cleaning
- Remove or Fill Missing Values: Identify and handle missing values appropriately.
- Correct Misplaced Data: Adjust any data that is not in the correct format or column.
- Standardize Formats: Ensure consistent data formats (e.g., date formats, text casing).
## 3. Export Cleaned Data:

Save the cleaned and structured data into a CSV file for further analysis.

In [75]:
import pandas as pd
import numpy as np
import re
import os
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)
#show all files available in Raw_data
path = 'F:/study_data/Diagnostic-task5/Raw_data/'
dfs = os.listdir(path)
df_og = pd.DataFrame()
for name in dfs:
    df_temp = pd.read_csv(path +name)
    df_og = pd.concat([df_og,df_temp])

del df_temp
del dfs
df_og = df_og.reset_index()
df = df_og.copy()

c:\Users\hamy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0,10,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [111]:
df_og.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94080 entries, 0 to 94079
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          94080 non-null  int64  
 1   Email          38554 non-null  object 
 2   Type           94080 non-null  object 
 3   Name           85229 non-null  object 
 4   Title          80864 non-null  object 
 5   MA URL         79881 non-null  object 
 6   MA Referrer    61136 non-null  object 
 7   ma_path        79837 non-null  object 
 8   IP Address     80364 non-null  object 
 9   cuid           61617 non-null  object 
 10  Date           69832 non-null  object 
 11  Message Id     5562 non-null   object 
 12  Template Id    1473 non-null   object 
 13  List Id        4628 non-null   float64
 14  Form Id        0 non-null      float64
 15  Campaign Id    0 non-null      float64
 16  Campaign Name  0 non-null      float64
 17  Scenario Id    4024 non-null   float64
 18  URL   


Let's clean the data. First, we are going to start with the null values in each columns.\
Only the Type column doesn't having missing values, while all the other columns has missing values.

## Cleaning Email column

Find out wrong format emails, delete the ones with wrong format and fix the ones with typo

In [113]:
df[df['Email'].str.split('.').str[-1].isin(['2611', 'cm', 'con', 'net', 'co'])]['Email'].values
#drop the ones ending with 2611, replace all emails ending with cm, con, co to com. retain the ones ending with net

array(['anh@nguyenngoc.2611', 'anhmiu2000@gmail.cm',
       'baothoa.hvtc.63@gmail.con', 'dangthuyhang1201@gmail.con',
       'hu.nguyen.21@saigontech.net', 'huongtran562005@gmail.con',
       'jannguyen1607@gmail.con', 'toyen301099@gmail.con',
       'tranlamnhatmai@gmail.con', 'yenhoangxuan1809@gmail.co',
       'toyen301099@gmail.con', 'tranlamnhatmai@gmail.con',
       'jannguyen1607@gmail.con', 'huongtran562005@gmail.con',
       'hu.nguyen.21@saigontech.net', 'dangthuyhang1201@gmail.con',
       'baothoa.hvtc.63@gmail.con', 'anhmiu2000@gmail.cm',
       'anh@nguyenngoc.2611', 'yenhoangxuan1809@gmail.co',
       'toyen301099@gmail.con', 'tranlamnhatmai@gmail.con',
       'jannguyen1607@gmail.con', 'huongtran562005@gmail.con',
       'hu.nguyen.21@saigontech.net', 'dangthuyhang1201@gmail.con',
       'baothoa.hvtc.63@gmail.con', 'anhmiu2000@gmail.cm',
       'anh@nguyenngoc.2611'], dtype=object)

In [78]:
def correct_email_endings(email):
    if isinstance(email, str):
        email = re.sub(r'\.cm$', '.com', email)
        email = re.sub(r'\.con$', '.com', email)
        email = re.sub(r'\.co$', '.com', email)
        return email
    return email

# Apply function to email column
df['Email_cleaned'] = df['Email'].apply(correct_email_endings)

# Function to delete emails ending with '2611'
def delete_specific_emails(email):
    if isinstance(email, str) and email.endswith('2611'):
        return np.nan
    return email

# Apply function to email column
df['Email_cleaned'] = df['Email_cleaned'].apply(delete_specific_emails)

In [79]:
df['Email_cleaned'].str.split('.').str[-1].dropna().unique()

array(['com', 'vn', 'net'], dtype=object)

In [116]:
def contains_specific_domains(s):
    if isinstance(s, str):
        return bool(re.search(r'@[a-zA-Z0-9_.]+\.(com|vn|net)$', s))
    return False
# Iterate over all columns except 'email'
for col in df.columns:
    if (col != 'Email') & (col != 'Email_cleaned'):
        mask = df['Email_cleaned'].isnull() & df[col].apply(contains_specific_domains)
        
        # Replace null values in Email column with the values from the current column
        df.loc[mask, 'Email_cleaned'] = df.loc[mask, col]
        if len(df.loc[mask])>0:
            print(col, " has some email address")


IP Address  has some email address


We managed to retrieve stray Email address from the IP Address column. For the other emails that are missing, it is possible that the user is not logged in so we can't know their email address and not because there's data error.

## Cleaning IP Address

Like in the email column, IP Address may have values from other columns that was wrongly categorized as IP address, as well as stray values that got mistakenly put into other columns as well. We are going to investigate IP Address column to see which values was wrongly categorized as well as take back IP address values from other columns

In [118]:
def is_valid_ip(s):
    if isinstance(s, str):
        return bool(re.match(r'^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$', s))
    return False

# Apply function to ip_address column to delete invalid IP addresses
df['ip_address_cleaned'] = df['IP Address'].apply(lambda x: x if is_valid_ip(x) else np.nan)


In [119]:
def is_valid_ip(s):
    if isinstance(s, str):
        return bool(re.match(r'^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$', s))
    return False
# Iterate over all columns except 'email'
for col in df.columns:
    if (col != 'IP Address') & (col != 'ip_address_cleaned'):
        mask = df['ip_address_cleaned'].isnull() & df[col].apply(is_valid_ip)
        
        # Replace null values in Email column with the values from the current column
        df.loc[mask, 'ip_address_cleaned'] = df.loc[mask, col]
        if len(df.loc[mask])>0:
            print(col, " has some IP address")




cuid  has some IP address
Date  has some IP address


## Cleaning Date

In [13]:
df['Date']

0        14-08-2021 16:56:41
1        14-08-2021 12:56:00
2        14-08-2021 12:55:56
3        14-08-2021 12:55:56
4        18-08-2021 16:14:06
                ...         
94075         12/8/2021 0:09
94076         12/8/2021 0:08
94077         12/8/2021 0:08
94078         12/8/2021 0:06
94079         12/8/2021 0:06
Name: Date, Length: 94080, dtype: object

In [120]:
df['date_column_cleaned'] = pd.to_datetime(df['Date'], errors='coerce')

# Identify rows where conversion failed (i.e., the value is NaT in the converted column)
peculiar_values = df[df['date_column_cleaned'].isna()]['Date']
peculiar_values=peculiar_values.dropna()

In [121]:
peculiar_values.astype(str).apply(lambda x:len(x)).unique()

array([ 2, 36, 13, 10, 14, 12, 15, 11], dtype=int64)

In [122]:
df['date_column_cleaned']

0       2021-08-14 16:56:41
1       2021-08-14 12:56:00
2       2021-08-14 12:55:56
3       2021-08-14 12:55:56
4       2021-08-18 16:14:06
                ...        
94075   2021-12-08 00:09:00
94076   2021-12-08 00:08:00
94077   2021-12-08 00:08:00
94078   2021-12-08 00:06:00
94079   2021-12-08 00:06:00
Name: date_column_cleaned, Length: 94080, dtype: datetime64[ns]

Looks like the peculiar values contains the cuid and some integers.

Let's find out if their are dates that got mixed up inside other columns. 

We know that a date 

In [123]:
date_patterns = [
    re.compile(r'\d{2}-\d{2}-\d{4} \d{2}:\d{2}:\d{2}'),  # 18-08-2021 16:14:06
    re.compile(r'\d{1,2}/\d{1,2}/\d{4} \d{1,2}:\d{2}')   # 12/8/2021 0:06
]

def contains_date(s):
    if isinstance(s, str):
        for pattern in date_patterns:
            if pattern.search(s):
                return True
    return False

for col in df.columns:
    if (col != 'Date') & (col != 'date_column_cleaned'):
        mask = df['date_column_cleaned'].isnull() & df[col].apply(contains_date)
        
        # Replace null values in Email column with the values from the current column
        df.loc[mask, 'date_column_cleaned'] = df.loc[mask, col]
        if len(df.loc[mask])>0:
            print(col, " has some Date")

IP Address  has some Date
cuid  has some Date
Message Id  has some Date
Template Id  has some Date


In [124]:
df['date_column_cleaned'] = pd.to_datetime(df['date_column_cleaned'])

### CUID

In [125]:
def is_valid_cuid(s):
    if isinstance(s, str):
        return bool(re.match(r'\b[0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{12}\b', s))
    return False

# Apply function to ip_address column to delete invalid IP addresses
df['cuid_cleaned'] = df['cuid'].apply(lambda x: x if is_valid_cuid(x) else np.nan)


In [126]:

# Iterate over all columns except 'email'
for col in df.columns:
    if (col != 'cuid') & (col != 'cuid_cleaned'):
        # Identify rows where Email is null and the current column contains '@'
        mask = df['cuid_cleaned'].isnull() & df[col].apply(is_valid_cuid)
        
        # Replace null values in Email column with the values from the current column
        df.loc[mask, 'cuid_cleaned'] = df.loc[mask, col]
        if len(df.loc[mask])>0:
            print(col, " has some cuid")




Date  has some cuid
Message Id  has some cuid


In [127]:
df['cuid_cleaned'].dropna().apply(lambda x:len(x)).unique()

array([36], dtype=int64)

In [24]:
df.head()

,index,Email,Type,Name,Title,MA URL,MA Referrer,ma_path,IP Address,cuid,Date,Message Id,Template Id,List Id,Form Id,Campaign Id,Campaign Name,Scenario Id,URL,Link,Tag,Email_cleaned,ip_address_cleaned,date_column_cleaned,cuid_cleaned
0,0,01202743441vy@gmail.com,templates_open,young-talent-reactivate,NaN,NaN,NaN,NaN,01202743441vy@gmail.com,NaN,14-08-2021 16:56:41,<202108140556.49300678081@smtp-relay.mailin.fr>,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[""young-talent-reactivate""]",01202743441vy@gmail.com,NaN,2021-08-14 16:56:41,NaN
1,1,01202743441vy@gmail.com,list_addition,NaN,NaN,NaN,NaN,NaN,01202743441vy@gmail.com,NaN,14-08-2021 12:56:00,NaN,NaN,40.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,01202743441vy@gmail.com,NaN,2021-08-14 12:56:00,NaN
2,2,030234180031@st.buh.edu.vn,list_addition,NaN,NaN,NaN,NaN,NaN,030234180031@st.buh.edu.vn,NaN,14-08-2021 12:55:56,NaN,NaN,40.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,030234180031@st.buh.edu.vn,NaN,2021-08-14 12:55:56,NaN
3,3,030235190162@st.buh.edu.vn,list_addition,NaN,NaN,NaN,NaN,NaN,030235190162@st.buh.edu.vn,NaN,14-08-2021 12:55:56,NaN,NaN,40.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,030235190162@st.buh.edu.vn,NaN,2021-08-14 12:55:56,NaN
4,4,030334180112@st.buh.edu.vn,page,KHÓA HỌC YOUNG TALENT,KHÓA HỌC YOUNG TALENT,https://uniace.vn/product/young-talent/,NaN,/product/young-talent/,030334180112@st.buh.edu.vn,13339736-d7ae-47a9-83bc-bf0f2969091e,18-08-2021 16:14:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,030334180112@st.buh.edu.vn,NaN,2021-08-18 16:14:06,13339736-d7ae-47a9-83bc-bf0f2969091e


In [128]:
df.to_csv('F:/study_data/Diagnostic-task5/uniace_og.csv')

In [151]:
df= pd.read_csv('F:/study_data/Diagnostic-task5/uniace_og.csv',index_col =0)


c:\Users\hamy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Identify the emails that are from students

In [152]:
df['email_suffix'] = df['Email_cleaned'].str.split('@').str[1]
df['email_prefix'] = df['Email_cleaned'].str.split('@').str[0]

In [153]:
df['is_schoolmail'] = df['email_suffix'].apply(lambda x: 1 if (isinstance(x,str) and (
                                                                                      ('.edu' in x) 
                                                                                      or ('udn' in x) 
                                                                                      or ('vanlanguni' in x))) 
                                                                                      else 0
                                                                                      )

Get schools that the user goes to based on email

In [154]:
df['school'] = np.where((df['is_schoolmail']==1) & (df['email_suffix']).str.contains('.edu'),
                        df['email_suffix'].str.split('.edu').str[0].str.split('.').str[-1], 
                        np.where((df['is_schoolmail']==1) & (df['email_suffix'].str.contains('vanlanguni')),
                                 'vanlang',
                                 np.where((df['is_schoolmail']==1) & (df['email_suffix'].str.contains('udn')),
                                                    df['email_suffix'].str.split('.udn').str[0].str.split('.').str[-1],
                                                    np.NaN)))

## Get product category

In [63]:
df['product_cat'] = df['MA URL'].str.split('product-category').str[1].str.strip('/')

In [71]:
df.groupby(['date_column_cleaned','product_cat']).count()['product'].reset_index()

,date_column_cleaned,product_cat,product
0,2021-01-08 02:18:00,all-level,1
1,2021-01-08 08:36:00,all-level,1
2,2021-01-08 12:45:00,mindset,1
3,2021-01-08 16:07:00,all-level,1
4,2021-01-08 16:07:00,power-query,1
...,...,...,...
1578,2021-12-08 23:04:00,career-coach,3
1579,2021-12-08 23:04:00,khac,3
1580,2021-12-08 23:14:00,programing-language,3
1581,2021-12-08 23:25:00,mindset,3
